In [1]:
import networkx as nx
from bertopic import BERTopic
import matplotlib.pyplot as plt

c:\Users\edupo\.conda\envs\recinfo\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\edupo\.conda\envs\recinfo\Lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\edupo\.conda\envs\recinfo\Lib\site-packages\umap\distances.py:1086: NumbaDeprecatio

In [2]:
from os import walk
lista_jogos = []

filenames = next(walk("../Dados/bertopics/"),
                 (None, None, []))[2]  # [] if no file
final = []
soma = 1
for i in filenames:
    lista_jogos.append(i)

In [3]:
print(lista_jogos)

['modelo_ark survival evolved', 'modelo_astroneer', 'modelo_dead by daylight', 'modelo_divinity original sin 2  definitive edition', 'modelo_dont starve together', 'modelo_euro truck simulator 2', 'modelo_factorio', 'modelo_garrys mod', 'modelo_grand theft auto v', 'modelo_human fall flat', 'modelo_insurgency sandstorm', 'modelo_left 4 dead 2', 'modelo_monster hunter world', 'modelo_playerunknowns battlegrounds', 'modelo_resident evil 2', 'modelo_rimworld', 'modelo_rocket league', 'modelo_rust', 'modelo_sid meiers civilization vi', 'modelo_slay the spire', 'modelo_stardew valley', 'modelo_subnautica', 'modelo_terraria', 'modelo_the elder scrolls v skyrim special edition']


In [4]:
# lista_jogos = lista_jogos[:6]
lista_jogos

['modelo_ark survival evolved',
 'modelo_astroneer',
 'modelo_dead by daylight',
 'modelo_divinity original sin 2  definitive edition',
 'modelo_dont starve together',
 'modelo_euro truck simulator 2',
 'modelo_factorio',
 'modelo_garrys mod',
 'modelo_grand theft auto v',
 'modelo_human fall flat',
 'modelo_insurgency sandstorm',
 'modelo_left 4 dead 2',
 'modelo_monster hunter world',
 'modelo_playerunknowns battlegrounds',
 'modelo_resident evil 2',
 'modelo_rimworld',
 'modelo_rocket league',
 'modelo_rust',
 'modelo_sid meiers civilization vi',
 'modelo_slay the spire',
 'modelo_stardew valley',
 'modelo_subnautica',
 'modelo_terraria',
 'modelo_the elder scrolls v skyrim special edition']

In [5]:
model = BERTopic.load("../Dados/bertopics/modelo_astroneer")

In [6]:
topico_info = model.get_topic(0)
str(topico_info)

"[('explore', 0.44539207), ('gameplay', 0.44502413), ('exploration', 0.42545074), ('exploring', 0.39940536), ('planet', 0.36397862), ('games', 0.3623904), ('playing', 0.35624903), ('game', 0.34682462), ('multiplayer', 0.33671063), ('play', 0.33442596)]"

In [7]:
def construir_nos_topicos(nome_jogo, modelo, grafo):
    
    for topico in range(len(modelo.get_topic_info())):
        nome_do_no = nome_jogo + '_topico' + str(topico)
        grafo.add_node(nome_do_no)

        grafo.nodes[nome_do_no]['topic_info'] = str(modelo.get_topic(topico))

In [8]:
import numpy as np
def calcular_distancia_entre_jogos(jogo_a,jogo_b, modelo_a, modelo_b):
    # modelo_a = BERTopic.load(f"../Dados/bertopics/modelo_{jogo_a}")
    # modelo_b = BERTopic.load(f"../Dados/bertopics/modelo_{jogo_b}")

    from sklearn.metrics.pairwise import cosine_similarity
    sim_matrix = cosine_similarity(modelo_a.topic_embeddings_, modelo_b.topic_embeddings_)
    
    def most_similar_topic(topic):
        topic = topic[1:]
        return np.argpartition(topic,-2)[-2:] 
        #TO-DO: VERIFICAR qntd de tópicos por jogo, para saber qnts tópicos similares retornar!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    
    maiores = np.apply_along_axis(most_similar_topic,1,sim_matrix)
    jogos = []
    linha = len(maiores)
    primeiro = True
    for (linha,valores) in zip(list(range(linha)),maiores):
        if primeiro:
            primeiro=False
            continue
        for coluna in valores:
            # print(linha,coluna)
            tmp = sim_matrix[linha,coluna+1]
            # jogos.append((f"{jogo_a}_topico{linha-1}",f"{jogo_b}_topico{coluna}",{"weight":tmp}))
            jogos.append((f"{jogo_a}_topico{linha-1}",f"{jogo_b}_topico{coluna}",{"weight":str(tmp)}))
    return jogos

In [9]:
import numpy as np

def calcular_distancia_entre_jogos(jogo_a, jogo_b, modelo_a, modelo_b):
    from sklearn.metrics.pairwise import cosine_similarity
    sim_matrix = cosine_similarity(modelo_a.topic_embeddings_, modelo_b.topic_embeddings_)

    def most_similar_topic(topic):
        topic = topic[1:]
        kth = min(len(topic) - 1, 5)  # Limitar kth ao tamanho do array - 1
        return np.argpartition(topic, -kth)[-kth:]

    maiores = np.apply_along_axis(most_similar_topic, 1, sim_matrix)
    jogos = []
    linha = len(maiores)
    primeiro = True
    for (linha, valores) in zip(list(range(linha)), maiores):
        if primeiro:
            primeiro = False
            continue
        for coluna in valores:
            tmp = sim_matrix[linha, coluna + 1]
            jogos.append((f"{jogo_a}_topico{linha - 1}", f"{jogo_b}_topico{coluna}", {"weight": str(tmp)}))
    return jogos


In [10]:
analisados = []
grafo = nx.Graph()
contador = 0

for jogo_A in lista_jogos:

    contador +=1
    print('jogo A: ' + jogo_A)

    for jogo_B in lista_jogos:

        print('jogo B: ' + jogo_B)

        #se os dois jogos são iguais, ignora
        if jogo_A == jogo_B:
            continue

        #se o jogo_B já foi analisado alguma vez como o jogo_A, pode skipar, pra não repetir comparação
        if jogo_B in analisados:
            continue

        #carrega modelos
        model_A = BERTopic.load("../Dados/bertopics/" + jogo_A)
        model_B = BERTopic.load("../Dados/bertopics/" + jogo_B)

        #nome dos jogos
        nome_jogo_A = jogo_A.replace('modelo_', '')
        nome_jogo_B = jogo_B.replace('modelo_', '')

        #constroi os nós referentes aos tópicos de cada jogo
        construir_nos_topicos(nome_jogo_A, model_A, grafo)
        construir_nos_topicos(nome_jogo_B, model_B, grafo)

        #descobre similaridades para as arestas
        lista_arestas = calcular_distancia_entre_jogos(nome_jogo_A, nome_jogo_B, model_A, model_B)

        # print(lista_arestas)
        
        #constrói arestas
        grafo.add_edges_from(lista_arestas)
        # for aresta in lista_arestas:
        #     grafo.add_edge(aresta[0], aresta[1], weight=aresta[2]['weight'])



    #adiciona o jogo na lista de já analisados para evitar fazer comparações repetidas
    analisados.append(jogo_A)
    print(jogo_A)
    print('jogo a está no numero: ' + str(contador))



jogo A: modelo_ark survival evolved
jogo B: modelo_ark survival evolved
jogo B: modelo_astroneer
jogo B: modelo_dead by daylight
jogo B: modelo_divinity original sin 2  definitive edition
jogo B: modelo_dont starve together
jogo B: modelo_euro truck simulator 2
jogo B: modelo_factorio
jogo B: modelo_garrys mod
jogo B: modelo_grand theft auto v
jogo B: modelo_human fall flat
jogo B: modelo_insurgency sandstorm
jogo B: modelo_left 4 dead 2
jogo B: modelo_monster hunter world
jogo B: modelo_playerunknowns battlegrounds
jogo B: modelo_resident evil 2
jogo B: modelo_rimworld
jogo B: modelo_rocket league
jogo B: modelo_rust
jogo B: modelo_sid meiers civilization vi
jogo B: modelo_slay the spire
jogo B: modelo_stardew valley
jogo B: modelo_subnautica
jogo B: modelo_terraria
jogo B: modelo_the elder scrolls v skyrim special edition
modelo_ark survival evolved
jogo a está no numero: 1
jogo A: modelo_astroneer
jogo B: modelo_ark survival evolved
jogo B: modelo_astroneer
jogo B: modelo_dead by da

In [11]:
nx.write_gml(grafo, "../Dados/grafo.gml")

In [12]:
import re
import networkx as nx

# Função auxiliar para corrigir o atributo "weight"
def fix_weight_attribute(value):
    return float(re.findall(r"\d+\.\d+", value)[0])

# Ler o arquivo GML
G = nx.read_gml('../Dados/grafo.gml')

# Corrigir o atributo "weight" para cada aresta
for u, v, attrs in G.edges(data=True):
    if 'weight' in attrs:
        attrs['weight'] = fix_weight_attribute(attrs['weight'])

# Salvar o grafo corrigido em formato GML
nx.write_gml(G, '../Dados/grafo_corrigido.gml')

In [13]:
# # Configuração do layout do grafo
# pos = nx.spring_layout(grafo)

# # Desenho do grafo
# nx.draw(grafo, pos, with_labels=False)

# nx.draw(grafo)